In [2]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Load nifti file
nifti = nib.load("/path/to/file.nii.gz")
data = nifti.get_fdata()
voxel_sizes = nifti.header.get_zooms()


In [4]:
# Function to plot a single slice
def plot_slice(index):
    dx, dy = voxel_sizes[0], voxel_sizes[1]
    nx, ny = data.shape[0], data.shape[1]

    extent = [0, dx * nx, 0, dy * ny]  # physical dimensions in mm

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(np.rot90(data[:, :, index]), cmap='gray', interpolation='nearest', extent=extent)
    ax.set_title(f'Axial Slice {index}', fontsize=16)
    ax.axis('off')
    ax.set_aspect('equal')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.show()

# Create slider for axial slices
axial_slider = widgets.IntSlider(
    min=0, 
    max=data.shape[2]-1, 
    step=1, 
    value=data.shape[2] // 2,
    description='Slice'
)

# Interactive display
widgets.interact(plot_slice, index=axial_slider)

interactive(children=(IntSlider(value=256, description='Slice', max=511), Output()), _dom_classes=('widget-int…

<function __main__.plot_slice(index)>

In [5]:

# Function to plot a slice in any axis
def plot_slice(index, axis):
    fig, ax = plt.subplots(figsize=(8, 8))
    
    if axis == 'Axial':
        img = np.rot90(data[:, :, index])
        dx, dy = voxel_sizes[0], voxel_sizes[1]
        nx, ny = data.shape[0], data.shape[1]
    elif axis == 'Coronal':
        img = np.rot90(data[:, index, :])
        dx, dy = voxel_sizes[0], voxel_sizes[2]
        nx, ny = data.shape[0], data.shape[2]
    elif axis == 'Sagittal':
        img = np.flip(np.rot90(data[index, :, :]), axis=1)#np.rot90(data[index, :, :])
        dx, dy = voxel_sizes[1], voxel_sizes[2]
        nx, ny = data.shape[1], data.shape[2]
        
    extent = [0, dx * nx, 0, dy * ny]

    ax.imshow(img, cmap='gray', interpolation='nearest', extent=extent)
    ax.set_title(f'{axis} Slice {index}', fontsize=14)
    ax.axis('off')
    ax.set_aspect('equal')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.show()

# Widgets for axis selection and slider
axis_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='View:'
)

slice_slider = widgets.IntSlider(
    min=0, 
    max=data.shape[2]-1, 
    value=data.shape[2]//2, 
    description='Slice:'
)

# Function to update slider limits when axis changes
def update_slider(*args):
    if axis_dropdown.value == 'Axial':
        slice_slider.max = data.shape[2]-1
        slice_slider.value = data.shape[2] // 2
    elif axis_dropdown.value == 'Coronal':
        slice_slider.max = data.shape[1]-1
        slice_slider.value = data.shape[1] // 2
    elif axis_dropdown.value == 'Sagittal':
        slice_slider.max = data.shape[0]-1
        slice_slider.value = data.shape[0] // 2

axis_dropdown.observe(update_slider, 'value')

# Interactive display
ui = widgets.VBox([axis_dropdown, slice_slider])
out = widgets.interactive_output(plot_slice, {'index': slice_slider, 'axis': axis_dropdown})

display(ui, out)


Output()